# Binned fits

Binned models and data can be created in two ways:
- from an unbinned model to a binned model or an unbinned dataset to a binned dataset
- directly from a binned object

In [ ]:
import hist as hist
import mplhep
import numpy as np
import zfit

In [ ]:
from matplotlib import pyplot as plt

normal_np = np.random.normal(loc=2., scale=3., size=10000)

obs = zfit.Space("x", limits=(-10, 10))

mu = zfit.Parameter("mu", 1., -4, 6)
sigma = zfit.Parameter("sigma", 1., 0.1, 10)
model_nobin = zfit.pdf.Gauss(mu, sigma, obs)

data_nobin = zfit.Data.from_numpy(obs, normal_np)

loss_nobin = zfit.loss.UnbinnedNLL(model_nobin, data_nobin)

In [ ]:
# make binned
binning = zfit.binned.Regular(50, -8, 10, name="x")
obs_bin = zfit.Space("x", binning=binning)

data = data_nobin.to_binned(obs_bin)
model = zfit.pdf.BinnedFromUnbinnedPDF(model_nobin, obs_bin)
loss = zfit.loss.BinnedNLL(model, data)

## Minimization

Both loss look the same to a minimizer and from here on, the whole minimization process is the same.

The following is the same as in the most simple case.

In [ ]:
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(loss)

In [ ]:
result.hesse()
print(result)

## Plotting the PDF

Since both PDFs are histograms, they can both be converted to histograms and plotted.

Using the `to_hist` method of the model and the `BinnedData` respectively, the data can be converted to a histogram.

In [ ]:
model_hist = model.to_hist()

plt.figure()
mplhep.histplot(model_hist, density=1, label="model")
mplhep.histplot(data.to_hist(), density=1, label="data")
plt.legend()
plt.title("After fit")


## To and from histograms

zfit interoperates with the Scikit-HEP histogram packages [hist](https://hist.readthedocs.io/en/latest/) and
[boost-histogram](https://boost-histogram.readthedocs.io/en/latest/), most notably with the `NamedHist`
 (or `Hist` if axes have a name) class.

We can create a `BinnedData` from a `(Named)Hist` and vice versa.

In [ ]:
h = hist.Hist(hist.axis.Regular(bins=15, start=-8, stop=10, name="x"))
h.fill(x=normal_np)
mplhep.histplot(h)

In [ ]:
binned_data = zfit.data.BinnedData.from_hist(h)
binned_data

In [ ]:
# convert back to hist
h_back = binned_data.to_hist()

plt.figure()
mplhep.histplot(h, label="original")
mplhep.histplot(h_back, label="back", alpha=0.5)
plt.legend()

## Model from a histogram

With a binned dataset, we can directly create a model from it using

In [ ]:
azimov_data = model.to_binneddata()
sampled_data = model.sample(1000)

In [ ]:
# The exact histogram from the PDF
azimov_data

In [ ]:
# A sample from the histogram
sampled_data
